# C.7.15 라쏘 변수를 이용한 모델 재실행

In [23]:
import pandas as pd
import numpy as np
df = pd.read_csv('Lasso_select_ERP.csv')   
df.shape

(12417, 73)

In [24]:
# SimpleInputer의 add_indicator 옵션을 제거한 버전

data = df.drop(['EBizSystem2'], axis=1)   # 타겟변수를 제외한 입력변수를 data에 저장
target = df['EBizSystem2']                # 타겟변수만 target에 저장

# 50:50 data partition.
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split( 
    data, target, test_size=0.5, random_state=42)   # test_size=0.5임에 주의 

# interval 변수의 null value를 평균(mean)으로 impute. 
from sklearn.impute import SimpleImputer
imp= SimpleImputer(strategy = 'mean')  
X_train2= imp.fit_transform(X_train) 
X_test2= imp.fit_transform(X_test)   # X_train과 and X_test 둘 다 imputation 적용해야 함에 유의

print("X_train2 shape:", X_train2.shape) 
print("X_test2 shape:", X_test2.shape) 

X_train2 shape: (6208, 72)
X_test2 shape: (6209, 72)


In [25]:
# Neural Network 모델 (Default 모델)
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import accuracy_score
mlp = MLPClassifier(max_iter = 1000, random_state = 0)

In [8]:
# 참조 코딩
mlp = MLPClassifier(max_iter = 1000, random_state = 0)

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  

# StratifiedKFold의 random_state 옵션값을 특정 숫자(예: 0)로 고정
cross_validation = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
params = {'solver':['lbfgs'], 'alpha':[1],
         'activation':['tanh','relu', 'logistic']}

grid_mlp = GridSearchCV(mlp, param_grid=params, scoring='accuracy', cv=cross_validation, n_jobs=-1)
grid_mlp.fit(X_train2, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_mlp.best_score_))
print("GridSearchCV best parameter:", (grid_mlp.best_params_)) 

GridSearchCV max accuracy:0.66302
GridSearchCV best parameter: {'activation': 'tanh', 'alpha': 1, 'solver': 'lbfgs'}


C:\Users\jason\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [9]:
# 참조 코딩
mlp = MLPClassifier(max_iter = 1000, random_state = 0)

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  

# StratifiedKFold의 random_state 옵션값을 특정 숫자(예: 0)로 고정
cross_validation = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
params = {'solver':['sgd'], 'alpha':[1], 'max_iter':[1000],
         'activation':['tanh','relu', 'logistic']}

grid_mlp = GridSearchCV(mlp, param_grid=params, scoring='accuracy', cv=cross_validation, n_jobs=-1)
grid_mlp.fit(X_train2, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_mlp.best_score_))
print("GridSearchCV best parameter:", (grid_mlp.best_params_)) 

GridSearchCV max accuracy:0.72052
GridSearchCV best parameter: {'activation': 'logistic', 'alpha': 1, 'max_iter': 1000, 'solver': 'sgd'}


In [11]:
# 참조 코딩
mlp = MLPClassifier(max_iter = 1000, random_state = 0)

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  

# StratifiedKFold의 random_state 옵션값을 특정 숫자(예: 0)로 고정
cross_validation = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
params = {'solver':['adam'], 'alpha':[1],
         'activation':['tanh','relu', 'logistic']}

grid_mlp = GridSearchCV(mlp, param_grid=params, scoring='accuracy', cv=cross_validation, n_jobs=-1)
grid_mlp.fit(X_train2, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_mlp.best_score_))
print("GridSearchCV best parameter:", (grid_mlp.best_params_)) 

GridSearchCV max accuracy:0.71923
GridSearchCV best parameter: {'activation': 'tanh', 'alpha': 1, 'solver': 'adam'}


In [26]:
import time
start = time.time()

mlp = MLPClassifier(max_iter = 1000, random_state = 0)

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  

# StratifiedKFold의 random_state 옵션값을 특정 숫자(예: 0)로 고정
cross_validation = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
params = {'solver':['adam','lbfgs', 'sgd'], 'alpha':[0.0001, 0.001, 0.01, 0.1, 1],
          'max_iter':[1000],'activation':['tanh','logistic']}

grid_mlp = GridSearchCV(mlp, param_grid=params, scoring='accuracy', cv=cross_validation, n_jobs=-1)
grid_mlp.fit(X_train2, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_mlp.best_score_))
print("GridSearchCV best parameter:", (grid_mlp.best_params_)) 

end = time.time()
print(f"Runtime of the program is {end - start}")

GridSearchCV max accuracy:0.72052
GridSearchCV best parameter: {'activation': 'logistic', 'alpha': 1, 'max_iter': 1000, 'solver': 'sgd'}
Runtime of the program is 499.63975501060486


In [27]:
best_clf = grid_mlp.best_estimator_
pred = best_clf.predict(X_test2)
print("Accuracy on test set:{:.5f}".format(accuracy_score(y_test, pred)))

from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y_test,best_clf.predict_proba(X_test2)[:, 1])
print("ROC AUC on test set:{:.5f}".format(ROC_AUC))

Accuracy on test set:0.72411
ROC AUC on test set:0.78041


In [14]:
# KNN 모델 (Default 모델 with n_neighbors=3).
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3) # random_state 파라미터가 없음에 주의!

In [15]:
# GridSearchCV.

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  

# StratifiedKFold의 random_state 옵션값을 특정 숫자(예: 0)로 고정
cross_validation = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
params = {'n_neighbors': range(3, 51)}

grid_knn = GridSearchCV(knn, param_grid=params, scoring='accuracy', cv=cross_validation, n_jobs=-1)
grid_knn.fit(X_train2, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_knn.best_score_))
print("GridSearchCV best parameter:", (grid_knn.best_params_)) 

GridSearchCV max accuracy:0.68944
GridSearchCV best parameter: {'n_neighbors': 35}


In [16]:
best_clf = grid_knn.best_estimator_
pred = best_clf.predict(X_test2)
print("Accuracy on test set:{:.5f}".format(accuracy_score(y_test, pred)))

from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y_test,best_clf.predict_proba(X_test2)[:, 1])
print("ROC AUC on test set:{:.5f}".format(ROC_AUC))

Accuracy on test set:0.68578
ROC AUC on test set:0.74090


In [17]:
# SVM model (default 모델)
from sklearn.svm import SVC 
svm = SVC(kernel='rbf', C=1, gamma = 'auto', random_state=0, probability=True) 
                                                             # probability=True 에 주의

In [19]:
# 참조 코딩
import time
start = time.time()

# SVM model (default 모델)
svm = SVC(kernel='rbf', C=1, gamma = 'auto', random_state=0, probability=True) 
                                                             # probability=True 에 주의

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  

# StratifiedKFold의 random_state 옵션값을 특정 숫자(예: 0)로 고정
cross_validation = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
params = {'kernel':['rbf','sigmoid'], 'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
         'gamma':['scale','auto']}

grid_svm = GridSearchCV(svm, param_grid=params, scoring='accuracy', cv=cross_validation, n_jobs=-1)
grid_svm.fit(X_train2, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_svm.best_score_))
print("GridSearchCV best parameter:", (grid_svm.best_params_))

end = time.time()
print(f"Runtime of the program is {end - start}")

GridSearchCV max accuracy:0.71795
GridSearchCV best parameter: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
Runtime of the program is 357.224942445755


In [20]:
# 참조 코딩
best_clf = grid_svm.best_estimator_
pred = best_clf.predict(X_test2)
print("Accuracy on test set:{:.5f}".format(accuracy_score(y_test, pred)))

from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y_test,best_clf.predict_proba(X_test2)[:, 1])
print("ROC AUC on test set:{:.5f}".format(ROC_AUC))

Accuracy on test set:0.72282
ROC AUC on test set:0.78370


In [21]:
import time
start = time.time()

# SVM model (default 모델)
svm = SVC(kernel='rbf', C=1, gamma = 'auto', random_state=0, probability=True) 
                                                             # probability=True 에 주의
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold  

# StratifiedKFold의 random_state 옵션값을 특정 숫자(예: 0)로 고정
cross_validation = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
params = {'kernel':['linear'], 'C':[0.0001, 0.001, 0.01, 0.1, 1, 10]} 
                                # C=100을 주면 프로그램 구동이 끝없이 이어져서 부득이 해당값은 배제  

grid_svm2 = GridSearchCV(svm, param_grid=params, scoring='accuracy', cv=cross_validation, n_jobs=-1)
grid_svm2.fit(X_train2, y_train)

print("GridSearchCV max accuracy:{:.5f}".format(grid_svm2.best_score_))
print("GridSearchCV best parameter:", (grid_svm2.best_params_))

end = time.time()
print(f"Runtime of the program is {end - start}")

GridSearchCV max accuracy:0.72713
GridSearchCV best parameter: {'C': 0.01, 'kernel': 'linear'}
Runtime of the program is 317.0663228034973


In [22]:
best_clf = grid_svm2.best_estimator_
pred = best_clf.predict(X_test2)
print("Accuracy on test set:{:.5f}".format(accuracy_score(y_test, pred)))

from sklearn.metrics import roc_auc_score
ROC_AUC = roc_auc_score(y_test,best_clf.predict_proba(X_test2)[:, 1])
print("ROC AUC on test set:{:.5f}".format(ROC_AUC))

Accuracy on test set:0.73297
ROC AUC on test set:0.78946
